In [15]:
import json
from mistral_common.tokens.tokenizers.mistral import MistralTokenizer
from mistral_common.protocol.instruct.messages import UserMessage
from mistral_common.protocol.instruct.request import ChatCompletionRequest

In [20]:


def contar_tokens_correctamente(ruta_archivo):
    """
    Lee un archivo JSONL y cuenta los tokens usando el método oficial
    `encode_chat_completion` de la biblioteca mistral-common.

    Args:
        ruta_archivo (str): La ruta al archivo .jsonl.
    """
    # Define el nombre del modelo que se usará tanto para cargar el tokenizador
    # como para crear la solicitud de chat.
    model_name = "mistral-small-latest" # o el modelo que uses

    try:
        tokenizer = MistralTokenizer.from_model(model_name)
    except Exception as e:
        print(f"Error al cargar el tokenizador: {e}")
        return

    print(f"Procesando archivo: {ruta_archivo} con el tokenizador de {model_name}\n")
    docs = []
    query = []
    answer = []
    with open(ruta_archivo, 'r', encoding='utf-8') as f:
        for i, linea in enumerate(f):
            try:
                dato = json.loads(linea.strip())

                # Extraer los textos de los campos requeridos
                texto_documento = dato.get("generated_document", {})[0]
                texto_query = dato.get("generated_query", {})[0]
                texto_respuesta = dato.get("generated_answer", {})[0]

                # --- LÓGICA DE TOKENIZACIÓN CORRECTA ---
                # Para cada texto, creamos una solicitud y la tokenizamos.

                # 1. Tokenizar el documento generado
                req_doc = ChatCompletionRequest(model=model_name, messages=[UserMessage(content=texto_documento)])
                tokens_documento = len(tokenizer.encode_chat_completion(req_doc).tokens)
                docs.append(tokens_documento)
                # 2. Tokenizar la query generada
                req_query = ChatCompletionRequest(model=model_name, messages=[UserMessage(content=texto_query)])
                tokens_query = len(tokenizer.encode_chat_completion(req_query).tokens)
                query.append(tokens_query)
                # 3. Tokenizar la respuesta generada
                req_resp = ChatCompletionRequest(model=model_name, messages=[UserMessage(content=texto_respuesta)])
                tokens_respuesta = len(tokenizer.encode_chat_completion(req_resp).tokens)
                answer.append(tokens_respuesta)
                # # Imprimir el resultado para la línea actual
                # print(f"Línea {i + 1}:")
                # print(f"  - Documento: {tokens_documento} tokens")
                # print(f"  - Query generada: {tokens_query} tokens")
                # print(f"  - Respuesta generada: {tokens_respuesta} tokens")

            except json.JSONDecodeError:
                print(f"Advertencia: La línea {i + 1} no es un JSON válido y será omitida.")
            except Exception as e:
                print(f"Error procesando la línea {i + 1}: {e}")

    print("\n--- Proceso completado ---")
    return {"gen_docs":docs, "gen_query":query, "gen_answer":answer}

In [22]:
# --- EJECUCIÓN DEL SCRIPT ---
# Reemplaza 'tu_archivo.jsonl' con el nombre de tu archivo
nombre_del_archivo_jsonl = '../doc_gen/fire/doc5/bge_large/generated_documents_nfcorpus.jsonl'
result_fire = contar_tokens_correctamente(nombre_del_archivo_jsonl)

Procesando archivo: ../doc_gen/fire/doc5/bge_large/generated_documents_nfcorpus.jsonl con el tokenizador de mistral-small-latest


--- Proceso completado ---


In [25]:
promedio = sum(result_fire['gen_docs']) / len(result_fire['gen_docs'])
print(f"El promedio de tokens es: {promedio}")

El promedio de tokens es: 357.70588235294116


In [26]:
promedio = sum(result_fire['gen_query']) / len(result_fire['gen_query'])
print(f"El promedio de tokens es: {promedio}")

El promedio de tokens es: 57.417956656346746


In [27]:
promedio = sum(result_fire['gen_answer']) / len(result_fire['gen_answer'])
print(f"El promedio de tokens es: {promedio}")

El promedio de tokens es: 177.03405572755418


## hyde

In [35]:


def contar_tokens_hyde_correctamente(ruta_archivo):
    """
    Lee un archivo JSONL y cuenta los tokens usando el método oficial
    `encode_chat_completion` de la biblioteca mistral-common.

    Args:
        ruta_archivo (str): La ruta al archivo .jsonl.
    """
    # Define el nombre del modelo que se usará tanto para cargar el tokenizador
    # como para crear la solicitud de chat.
    model_name = "mistral-small-latest" # o el modelo que uses

    try:
        tokenizer = MistralTokenizer.from_model(model_name)
    except Exception as e:
        print(f"Error al cargar el tokenizador: {e}")
        return

    print(f"Procesando archivo: {ruta_archivo} con el tokenizador de {model_name}\n")

    docs = []
    with open(ruta_archivo, 'r', encoding='utf-8') as f:
        for i, linea in enumerate(f):
            try:
                dato = json.loads(linea.strip())
                documents = dato.get("generated_documents", [])
                len_docs=0
                for j, doc in enumerate(documents):

                    # 1. Tokenizar el documento generado
                    req_doc = ChatCompletionRequest(model=model_name, messages=[UserMessage(content=doc)])
                    tokens_documento = len(tokenizer.encode_chat_completion(req_doc).tokens)
                    len_docs+=tokens_documento
                docs.append(len_docs)

            except json.JSONDecodeError:
                print(f"Advertencia: La línea {i + 1} no es un JSON válido y será omitida.")
            except Exception as e:
                print(f"Error procesando la línea {i + 1}: {e}")

    print("\n--- Proceso completado ---")
    return {"gen_docs":docs}

In [36]:
nombre_del_archivo_jsonl_hyde = '../doc_gen/hyde/generated_documents_nfcorpus.jsonl'
result_hyde = contar_tokens_hyde_correctamente(nombre_del_archivo_jsonl_hyde)

Procesando archivo: ../doc_gen/hyde/generated_documents_nfcorpus.jsonl con el tokenizador de mistral-small-latest


--- Proceso completado ---


In [37]:
promedio = sum(result_hyde['gen_docs']) / len(result_hyde['gen_docs'])
print(f"El promedio de tokens es: {promedio}")

El promedio de tokens es: 2387.2260061919505


In [39]:
nombre_del_archivo_jsonl_q2docs = '../doc_gen/query2doc/generated_documents_nfcorpus.jsonl'
result_q2d = contar_tokens_hyde_correctamente(nombre_del_archivo_jsonl_q2docs)

Procesando archivo: ../doc_gen/query2doc/generated_documents_nfcorpus.jsonl con el tokenizador de mistral-small-latest


--- Proceso completado ---


In [40]:
promedio = sum(result_q2d['gen_docs']) / len(result_q2d['gen_docs'])
print(f"El promedio de tokens es: {promedio}")

El promedio de tokens es: 444.05882352941177
